# Recommendation based on Genres and Descriptions

In [17]:
import numpy  as np
import pandas as pd
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import utils as u

In [19]:
df = pd.read_csv("Titles.csv")
credits_df = pd.read_csv("Credits.csv")

In [20]:
df['rating']=df['age_certification'].replace({
    'PG-13':'Teens - Age above 12',
    'TV-MA' :'Adults',
    'PG' : 'Kids - with parental guidance',
    'TV-14':'Teens Age above age of 14',
    'TV-PG':'Kids-with parental guidance',
    'TV-Y':'Kids',
    'TV-Y7': 'Kids-Age above 7',
    'R': 'Adults',
    'TV-G':'Kids',
    'NC-17': 'Adults',
    'NR': 'NR',
    'UR': 'UR',
})

df['imdb_score'].fillna(0, inplace=True)
df["production_countries"]=df["production_countries"].apply(u.repair_array_bound_categories)
df = df.explode('production_countries').drop_duplicates()

In [21]:
df["director"] = pd.merge(
    df, credits_df[credits_df["role"] == "DIRECTOR"], on="id", how="left"
)["name"].replace(np.nan, None)

df["actors"] = pd.merge(
    df,
    pd.merge(df, credits_df[credits_df["role"] == "ACTOR"], on="id", how="left")
    .groupby("id")["name"]
    .apply(lambda x: x.tolist() if x is not np.nan else None),
    on="id",
    how="left",
)["name"].apply(lambda x: ["" if i is np.nan else str(i) for i in x])

df["actors"] = df["actors"].replace(np.nan, "")

In [22]:
df["overview"] = (
    (
        df["title"].astype(str)
        + " "
        + df["description"].astype(str)
        + " "
        + df["genres"].apply(lambda x: " ".join(x))
        + " "
        + df["director"].astype(str)
        + " "
        + df["actors"].apply(lambda x: "" if x is [] else " ".join(x))
    )
    .str.lower()
    .str.replace("\n", " ")
    .str.replace("—", "")
)

In [23]:
df = df[df.type=='MOVIE'].reset_index()

In [24]:
count = CountVectorizer(stop_words="english", ngram_range=(1, 5))
count_matrix = count.fit_transform(df["overview"])
cosine_sim = cosine_similarity(count_matrix, count_matrix)
indices = pd.Series(df.index, index=df["title"])
count = CountVectorizer(stop_words="english", ngram_range=(1, 5))
count_matrix = count.fit_transform(df["overview"])
cosine_sim = cosine_similarity(count_matrix, count_matrix)
indices = pd.Series(df.index, index=df["title"])

In [25]:
df.to_parquet("data.parquet", compression='gzip')

In [26]:
cosine_sim = pd.DataFrame(cosine_sim)
cosine_sim.columns = cosine_sim.columns.astype(str)
cosine_sim.to_parquet("cosine_sim.parquet", compression='gzip')